In [1]:
from clearml.automation.controller import PipelineDecorator
from clearml import TaskTypes


@PipelineDecorator.component(
    return_values=["the_dataset"], cache=False, task_type=TaskTypes.data_processing
)
def make_new_dataset_component(project, i_dataset, num_samples_per_chunk=500):
    from image_classifier_training import pipeline_functions
    return pipeline_functions.make_new_dataset(project, i_dataset, num_samples_per_chunk=500)
    


@PipelineDecorator.component(
    return_values=["run_model_path", "run_tb_path"],
    cache=False,
    task_type=TaskTypes.training,
)
def train_image_classifier_component(
    clearml_dataset, backbone_name, run_model_uri, run_tb_uri, local_data_path="/data"
):
    from image_classifier_training import pipeline_functions

    return pipeline_functions.train_image_classifier(
        clearml_dataset,
        backbone_name,
        run_model_uri,
        run_tb_uri,
        local_data_path="/data",
    )


from clearml import Task
@PipelineDecorator.pipeline(
    name="fastai_image_classification_pipeline",
    project="lavi-testing",
    version="0.2",
    multi_instance_support=True,
)
def fastai_image_classification_pipeline(run_id, i_dataset, backbone_name="resnet34"):
    from clearml import Task
    from image_classifier_training import pipeline_functions
    class TaskURIs:
        def __init__(self, project, pipeline_name, run_id):
            path_pref = f"{project}/{pipeline_name}"
            self.tboard = f"{path_pref}/tboard/{run_id}"
            self.models = f"{path_pref}/models/{run_id}"
            self.evaluations = f"{path_pref}/evaluations/{run_id}"
    project_name = "lavi-testing"
    pipeline_name = "fastai_image_classification"
    

    pipeline_task = Task.current_task()
    print("pipeline task=", pipeline_task)
    config = {"run_id": run_id}
    config["backbone_name"] = backbone_name
    config["i_dataset"] = backbone_name

    print("run_id:", run_id)

    if pipeline_task:
        config = pipeline_task.connect_configuration(config, name="run_config")

    print("make dataset")
    training_dataset = make_new_dataset_component(
        project=project_name, i_dataset=i_dataset, num_samples_per_chunk=500
    )
    config["training_dataset"] = {
        "id": training_dataset.id,
        "name": training_dataset.name,
    }

    run_uris = TaskURIs(
        project=project_name, pipeline_name=pipeline_name, run_id=run_id
    )

    config["run_uris"] = json.loads(json.dumps(vars(run_uris), default=str))

    print("train model")
    run_model_path, run_tb_path = run_learner_path = train_image_classifier_component(
        clearml_dataset=training_dataset,
        backbone_name=backbone_name,
        run_model_uri=run_uris.models,
        run_tb_uri=run_uris.tboard,
        local_data_path="/data",
    )
    #     config["run_learner_path"] = str(run_learner_path)

    #     print("evaluate model")
    #     run_eval_path = eval_model(
    #         run_learner_path,
    #         run_dataset_path,
    #         run_splits_path,
    #         run_id,
    #         task_paths.evaluations_path,
    #     )
    #     config["run_eval_path"] = str(run_eval_path)
    #     # clearml_task.close()

    print("pipeline complete")

    return config

In [2]:
    
from datetime import datetime
run_id = f"run_{datetime.utcnow().strftime('%Y_%m_%dT%H_%M_%S.%f')[:-3]}"
PipelineDecorator.run_locally()
fastai_image_classification_pipeline(run_id = "run_1", i_dataset=0, backbone_name="resnet34")

ClearML Task: created new task id=95c0e17390a6402daa89bee651cc0d50
2022-07-09 20:19:29,808 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/cdcb102e595545cc90e370e977ced98a/experiments/95c0e17390a6402daa89bee651cc0d50/output/log
ClearML pipeline page: https://app.clear.ml/pipelines/cdcb102e595545cc90e370e977ced98a/experiments/95c0e17390a6402daa89bee651cc0d50
pipeline task= <clearml.task.Task object at 0x7f40e0d41790>
run_id: run_1
make dataset
Launching step [make_new_dataset_component]
ClearML results page: https://app.clear.ml/projects/cdcb102e595545cc90e370e977ced98a/experiments/afeca009b496475c9d7680ba7d5c1cdc/output/log


Traceback (most recent call last):
  File "/tmp/tmp1bq7ohox.py", line 25, in <module>
    results = make_new_dataset_component(**kwargs)
  File "/tmp/tmp1bq7ohox.py", line 6, in make_new_dataset_component
    from image_classifier_training import pipeline_functions
  File "/usr/local/lib/python3.8/dist-packages/clearml/binding/import_bind.py", line 54, in __patched_import3
    mod = builtins.__org_import__(
ModuleNotFoundError: No module named 'image_classifier_training'
Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/clearml/automation/controller.py", line 3532, in sanitized_env
    a_result = func(*a_args, **a_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/clearml/automation/controller.py", line 3302, in internal_

In [3]:
# class TaskURIs:
#     def __init__(self, project, pipeline_name, run_id):
#         path_pref = f"{project}/{pipeline_name}"
#         self.tboard = f"{path_pref}/tboard/{run_id}"
#         self.models = f"{path_pref}/models/{run_id}"
#         self.evaluations = f"{path_pref}/evaluations/{run_id}"

# from datetime import datetime
# pipeline_name = "fastai_image_classification"
# run_id = f"run_{datetime.utcnow().strftime('%Y_%m_%dT%H_%M_%S.%f')[:-3]}"
# run_uris= TaskURIs(project="lavi-tests", pipeline_name=pipeline_name, run_id=run_id)
# json.loads(json.dumps(vars(run_uris), default=str))

In [4]:
# !pip install -e ./image_classifier_training/